In [1]:
import numpy as np
import argparse
import c3.libraries.fidelities as fidelities
import utils
from utils import *
import c3.libraries.algorithms as algorithms
import c3.utils.qt_utils as qt_utils
import c3.libraries.constants as constants
from multi_level_qubits.TwoQubitsExperiment import TwoQubitsExperiment

2021-10-21 16:16:32.488842: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-21 16:16:32.488907: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
parser = argparse.ArgumentParser()
parser.add_argument("output", help="Output directory")
args = parser.parse_args()
output_dir = args.output
#output_dir = "./output_1"

In [3]:
def spectral_peaks_goal(propagators: dict, instructions: dict, index, dims,
                        qubit: chip.Qubit, generator: Generator, drive: chip.Drive,
                        n_eval=-1, num_peaks=2) -> tf.Tensor:
    # find energies of the qubit
    energies = qubit.get_Hamiltonian().numpy().diagonal().real
    vec1 = np.array([energies[3] - energies[2], energies[1] - energies[0]]) / (2 * np.pi)

    infids = []
    for gate, propagator in propagators.items():
        # create signal and find peaks
        instr = instructions[gate]
        signal = generator.generate_signals(instr)[drive.name]
        peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), num_peaks)
        peakFrequencies = tf.convert_to_tensor(peakFrequencies)
        peakValues = tf.convert_to_tensor(peakValues)

        # add frequency offsets to infidelity
        #infid = np.linalg.norm(vec1 - peakFrequencies)
        infid1 = tf.abs(vec1[0] - peakFrequencies[0]) / peakFrequencies[0]
        infid1 += tf.abs(vec1[1] - peakFrequencies[1]) / peakFrequencies[1]

        # add peak height difference to infidelity
        if peakFrequencies[1] > peakFrequencies[0]:
            a = peakValues[1]
            b = np.sqrt(2) * peakValues[0]
        else:
            a = peakValues[0]
            b = np.sqrt(2) * peakValues[1]
        infid2 = tf.abs(a - b) / tf.maximum(a, b)
        infid = infid1 * (1 + infid2)
        infids.append(infid)
    result = tf.reduce_mean(infids)
    print("type: ", type(result))
    return result


def printInformation(exper: TwoQubitsExperiment, name: str) -> None:
    signal = exper.generateSignal()
    peakFrequencies, peakValues = findFrequencyPeaks(signal["ts"].numpy(), signal["values"].numpy(), 4)
    print("peaks: ", np.sort(peakFrequencies))
    exper.saveSignal("signal_" + name)
    exper.plotSignal("signal_" + name)
    exper.saveTimeEvolution("populations_" + name)
    exper.plotTimeEvolution("populations_" + name)
    exper.savePropagator("propagator_" + name)
    exper.plotPropagator("propagator_" + name)

In [4]:
freq1 = 5e9
freq2 = 4e9
anharm = 300e6
t_final = 15e-9
sigma = t_final / 15
sigma2 = sigma / 2
relative_amp = 2
carrier_freq = 4.7e9
coupling_strength = 0 #20e6
optimisable_params = {
    "pwc": [
        "amp",
        "xy_angle", "freq_offset",
        "delta",
        "t_final", #"t_bin_start", "t_bin_end",
        "inphase"
    ],
    "carrier": [
        "freq",
        "framechange"
    ]
}

In [5]:
ideal_Unity = qt_utils.np_kron_n([constants.Id, constants.Id])
ideal_X90_q1 = qt_utils.np_kron_n([constants.x90p, constants.Id])
ideal_X90_q2 = qt_utils.np_kron_n([constants.Id, constants.x90p])
ideal_CNOT_q1 = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 0, 1], [0, 0, 1, 0]], dtype=np.complex128)
ideal_CNOT_q2 = constants.GATES["cx"]

In [6]:
envelope = createPWCDoubleGaussianPulse(t_final, sigma, sigma2, relative_amp, 30)

exper = TwoQubitsExperiment(directory=f"{output_dir}/")
exper.prepare(t_final, (freq1, freq2), (anharm, anharm), carrier_freq, envelope,
              ideal_gate=ideal_X90_q2,
              occupied_levels=[0],
              useDrag=True)

#exper.saveIdealPropagator(name="ideal_propagator")

# print information
energies = exper.getEnergies()
print("energies: ", energies)
print("energies differences: ", [(energies[i + 1] - energies[i]) / (2 * np.pi) for i in range(len(energies) - 1)])
printInformation(exper, "before")

# first optimisation
callback = lambda infid: print(infid)
params_after = exper.optimise(
    optimisable_params,
    algorithm=algorithms.cmaes,
    algorithm_options={
        "stop_at_convergence": 15,
        "spread": 0.2,
        "popsize": 25,
        #"maxfun": 200,
    },
    fidelity_function=fidelities.unitary_infid_set_with_penalty,
    fidelity_params={
        "active_levels": 4,
        #"qubit": exper.getQubit(),
        #"generator": exper.getExperiment().pmap.generator,
        #"drive": getDrive(exper.getExperiment().pmap.model, exper.getQubit())
        "penalty_factor": 1
    },
    callback=callback,
)
printInformation(exper, "after")
print(params_after)

2021-10-21 16:16:46.957103: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-10-21 16:16:46.957388: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2021-10-21 16:16:46.957409: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)
2021-10-21 16:16:46.957447: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (localhost.localdomain): /proc/driver/nvidia/version does not exist
2021-10-21 16:16:46.958365: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set


enabling DRAG2


2021-10-21 16:16:53.751786: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-10-21 16:16:53.768703: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2793760000 Hz


saving plot in ./output_1//ideal_propagator.png
saving plot in ./output_1//ideal_propagator_phases.png
energies:  [0.00000000e+00 2.51327412e+10 4.83805269e+10 6.97433569e+10
 8.92212314e+10 3.14159265e+10 5.65486678e+10 7.97964534e+10
 1.01159283e+11 1.20637158e+11 6.09468975e+10 8.60796387e+10
 1.09327424e+11 1.30690254e+11 1.50168129e+11 8.85929128e+10
 1.13725654e+11 1.36973440e+11 1.58336270e+11 1.77814144e+11
 1.14353973e+11 1.39486714e+11 1.62734499e+11 1.84097330e+11
 2.03575204e+11]
energies differences:  [4000000000.0, 3700000000.000001, 3399999999.9999986, 3100000000.000002, -9200000000.000002, 3999999999.9999995, 3700000000.0000024, 3399999999.9999986, 3100000000.000002, -9500000000.0, 4000000000.0000014, 3700000000.0, 3399999999.9999986, 3100000000.000002, -9800000000.000006, 4000000000.0000014, 3700000000.0, 3399999999.999996, 3100000000.0000043, -10100000000.0, 4000000000.0000014, 3700000000.0000024, 3399999999.9999986, 3099999999.9999995]
peaks:  [3.20106702e+09 4.20140

KeyboardInterrupt: 